In [1]:
import gym
import warnings
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Suppress DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow as tf

# List all GPUs available to TensorFlow
gpus = tf.config.list_physical_devices('GPU')

# Check if any GPUs are available
if len(gpus) > 0:
    print("GPU is available")
    for gpu in gpus:
        print(f"GPU device: {gpu}")
else:
    print("GPU is not available")

GPU is available
GPU device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [4]:
# Question 1: Cartpole-v0
env = gym.make(
    "CartPole-v0",
    # render_mode="human"
    )

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)


def CartPole_RL(model, discount_factor=0.95, num_choices=2):
    
    # Save rewards for visualization later
    episode_rewards = []

    for episode in range(1000):
        # print(f"CartPole-v0, episode {episode}")
        # Initiate one episode
        observation, info = env.reset()

        obs_history = []
        reward_history = []
        action_history = []

        terminated = False
        truncated = False



        # Roll out one episode
        while (not terminated) and (not truncated):

            # Make a prediction based on the state
            probabilities = model.predict(np.array([observation]), verbose=0)[0]
            # print(probabilities)

            # This line stochastically samples an action based on the probabilities
            action = np.random.choice(num_choices, p=probabilities)
            # print(action)

            obs_history.append(observation)
            action_history.append(action)

            observation, reward, terminated, truncated, info = env.step(action)
            print(observation, reward, terminated, truncated, info)

            reward_history.append(reward)
            


        
        # Discount rewards
        discounted_rewards = []
        cumulative_reward = 0
        for reward in reversed(reward_history):
            cumulative_reward = reward + discount_factor * cumulative_reward
            discounted_rewards.insert(0, cumulative_reward)

        # print(f"Cumulative Reward History: {discounted_rewards}")


        # print(f"Frame: {count_frame}")
        # print(f"Obs History: {obs_history}")
        # print(f"Reward History: {reward_history}")
        # print(f"Action History: {action_history}")



        def adjust_weights(obs_history, action_history, discounted_rewards):
            
            with tf.GradientTape() as tape:
                # Predict action probabilities
                action_probabilities = model(np.array(obs_history))
                # print(action_probabilities)

                # Get probabilities of actions that were taken
                indices = list(zip(range(len(action_history)), action_history))
                # print(indices)
                chosen_action_probs = tf.gather_nd(action_probabilities, indices)
                # print(chosen_action_probs)

                # Compute loss
                loss = -tf.math.log(chosen_action_probs) * discounted_rewards
                loss = tf.reduce_mean(loss)

                
            # Calculate gradients
            grads = tape.gradient(loss, model.trainable_variables)
            # print(grads)
            
            # Apply gradients to model weights
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # # Print updated model weights
            # for var in model.trainable_variables:
            #     print(var.name, var.numpy()[1])


        adjust_weights(obs_history=obs_history, action_history=action_history, discounted_rewards=discounted_rewards)


        # Append episode rewards for plotting
        total_reward = sum(reward_history)
        episode_rewards.append(total_reward)
        print(f"CartPole-v0 episode {episode}, reward sum: {total_reward}")

        # clear_output(wait=True)
        
    env.close()

    # After the for loop
    plt.plot(episode_rewards)
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.show()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation="relu", input_shape=(4,)),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(2, activation="softmax")
])

CartPole_RL(model)

### Question 2 (Pong)

In [2]:
import gym
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Suppress DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def check_gpu_availability():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        print("GPU is available")
        for gpu in gpus:
            print(f"GPU device: {gpu}")
    else:
        print("GPU is not available")

check_gpu_availability()

GPU is available
GPU device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [21]:
processed_observation[1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.])

In [19]:
def preprocess(image, downsample_factor=3):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 2D float array """
    image = image[35:195] # crop
    image = image[::downsample_factor,::downsample_factor,0] # downsample by factor of 2
    image[image == 144] = 0 # erase background (background type 1)
    image[image == 109] = 0 # erase background (background type 2)
    image[image != 0] = 1 # everything else (paddles, ball) just set to 1
    # In preprocess function
    return np.reshape(image.astype(np.float64).ravel(), [80, 80])


env = gym.make("Pong-v0", 
            #    render_mode='human'
               )

# Roll out 10 episdoes
for episode in range(1):
    print(f"Pong-v0, episode {episode}")
    # Initiate one episode
    observation, info = env.reset()

    obs_history = []
    reward_history = []
    action_history = []

    terminated = False
    truncated = False

    # Roll out one episode
    while (not terminated) and (not truncated):
        
        # Policy (Action)
        action = env.action_space.sample()
        action_history.append(action)

        # Preprocess observation
        processed_observation = preprocess(observation)
        obs_history.append(processed_observation)

        observation, reward, terminated, truncated, info = env.step(action)

        print(processed_observation.shape, reward, terminated, truncated, info)
        
        reward_history.append(reward)
        

env.close()

Pong-v0, episode 0
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 2, 'frame_number': 2}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 6, 'frame_number': 6}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 9, 'frame_number': 9}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 11, 'frame_number': 11}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 13, 'frame_number': 13}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 15, 'frame_number': 15}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 19, 'frame_number': 19}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 22, 'frame_number': 22}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 26, 'frame_number': 26}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 30, 'frame_number': 30}
(40, 40) 0.0 False False {'lives': 0, 'episode_frame_number': 33, 'frame_number': 33}
(40, 40) 0.0 False False {'lives': 0, 'ep

In [10]:
def preprocess(image, downsample_factor=3):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 2D float array """
    image = image[35:195] # crop
    image = image[::downsample_factor,::downsample_factor,0] # downsample by factor
    image[image == 144] = 0 # erase background (background type 1)
    image[image == 109] = 0 # erase background (background type 2)
    image[image != 0] = 1 # everything else (paddles, ball) just set to 1
    # In preprocess function
    return np.reshape(image.astype(np.float64).ravel(), [80, 80])

def build_model(input_shape=(80, 80, 1), num_choices=6, reg=0.0001):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(reg), input_shape=input_shape),
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu", kernel_regularizer=tf.keras.regularizers.l2(reg)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_choices, activation="softmax")
    ])
    return model

def select_action(model, observation, num_choices=6):
    probabilities = model.predict(np.array([observation]), verbose=0)[0]
    # print(probabilities)
    return np.random.choice(num_choices, p=probabilities)

def compute_discounted_rewards(reward_history, discount_factor=0.99):
    discounted_rewards, cumulative_reward = [], 0
    for reward in reversed(reward_history):
        cumulative_reward = reward + discount_factor * cumulative_reward
        discounted_rewards.insert(0, cumulative_reward)
    return discounted_rewards

def adjust_weights(model, optimizer, obs_history, action_history, discounted_rewards):
    with tf.GradientTape() as tape:
        probs=model(np.array(obs_history))
        indices=tf.stack([tf.range(len(action_history),dtype=tf.int32),tf.convert_to_tensor(action_history,dtype=tf.int32)],axis=1)
        chosen_probs=tf.gather_nd(probs,indices)
        loss=-tf.math.log(chosen_probs)*discounted_rewards
        loss=tf.reduce_mean(loss)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))

def Pong_RL():
    env = gym.make("Pong-v0")
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
    model = build_model()
    episode_rewards = []
    episode = 0

    consecutive_21_rewards = 0  # Count number of 21 occurences
    should_train = True  # Initialize flag for training

    while True:
        observation, info = env.reset()
        obs_history, reward_history, action_history = [], [], []

        terminated = False
        truncated = False

        while not terminated and not truncated:
            processed_observation = preprocess(observation)
            action = select_action(model, processed_observation)
            obs_history.append(processed_observation)
            action_history.append(action)

            observation, reward, terminated, truncated, info = env.step(action)
            reward_history.append(reward)

        discounted_rewards = compute_discounted_rewards(reward_history)

        total_reward = sum(reward_history)
        episode_rewards.append(total_reward)

        moving_num, window = 21, 100
        if episode >= window-1:
            moving_avg = np.mean(episode_rewards[-window:])
            print(f"Pong-v0 episode {episode}, reward sum: {total_reward}, last {window} avg: {moving_avg:.2f}")
            
            if moving_avg > moving_num:
                print(f"Stopping as the last {window}-episode moving average is greater than {moving_num}")
                break
        else:
            print(f"Pong-v0 episode {episode}, reward sum: {total_reward}")

        ### TRAINING STOP

        if total_reward == 21:  # Check for consecutive rewards of 21
            consecutive_21_rewards += 1
            if consecutive_21_rewards >= 10 and should_train == True:
                print("Stopping training as the reward has been 21 for 10 episodes in a row")
                should_train = False  # Set flag to False

                # Check if folder exists, if not create it
                if not os.path.exists("saved_model"):
                    os.makedirs("saved_model")

                # Save the model
                model.save("saved_model/pong_model")
                
        else:
            consecutive_21_rewards = 0  # Reset the counter if the reward is not 21

        # Modify the weight adjustment to respect the should_train flag
        if should_train:
            adjust_weights(model, optimizer, obs_history, action_history, discounted_rewards)

        episode += 1

    env.close()

    # Save Results
    if not os.path.exists('artifacts'):
        os.makedirs('artifacts')
    plt.plot(episode_rewards)
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.savefig('artifacts/pong_rewards.png')

if __name__ == "main":
    check_gpu_availability()
    Pong_RL()

In [11]:
Pong_RL()

c:\Users\nuke2\Desktop\NW Work\Fall_02 Work\MSIA-FQ2\.venv\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(


ValueError: cannot reshape array of size 2916 into shape (80,80)